In [1]:
import pandas as pd
import numpy as np

In [2]:
# define data paths
stock_values_path = "agg.csv"
compustat_path = "./archive/compustat_quarterly.csv"
wrds_ratios_path = "./archive/wrds_quarterly_ratios.csv"

In [4]:
# look at stock price
stock_values = pd.read_csv("agg.csv", parse_dates=["datadate"], index_col="index")
stock_values.head(2)

,datadate,prcod,prchd,prcld,prccd,cusip
index,,,,,,
0,2004-09-01,2.12,2.57,1.93,2.55,665575106
1,2004-10-01,2.55,2.95,2.50,2.69,665575106


In [6]:
# look at stock performance data
compustat = pd.read_csv(compustat_path, parse_dates=["datadate"])
compustat.head(2)

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsp12,xoptepsq,xoptepsqp,xoptq,xoptqp,xrdq,xsgaq,costat,dvpspq,adjex
0,1004,2010-02-28,2009,3.0,5,INDL,C,D,STD,AIR,...,0.0,0.0,0.0,0.0,0.0,NaN,34.091,A,0.0,1.0
1,1004,2010-05-31,2009,4.0,5,INDL,C,D,STD,AIR,...,0.0,0.0,0.0,0.0,0.0,NaN,40.177,A,0.0,1.0


In [8]:
# look at fundamental stock data
wrds_ratios = pd.read_csv(wrds_ratios_path, parse_dates=["qdate"])
wrds_ratios.head(2)

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,sale_nwc,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,TICKER,cusip
0,1004,54594,2009-05-31,2009-11-30,2010-01-31,17.934,1.077,7.981,14.758,15.243,...,2.257,0.0,0.0,0.0,-0.043,1.152,2.250,NaN,AIR,00036110
1,1004,54594,2009-05-31,2009-11-30,2010-02-28,17.554,1.077,7.981,14.446,14.921,...,2.257,0.0,0.0,0.0,-0.043,1.128,2.203,NaN,AIR,00036110


In [9]:
# create bins for each quarter, give each bin an ID
start_time = pd.Timestamp('2009-01-01 00:00:00')
stop_time = pd.Timestamp('2023-01-01 00:00:00')
interval = pd.Interval(start_time, start_time, closed='left')
bin_edges = [start_time]
while bin_edges[-1] < stop_time:
    bin_edges.append(bin_edges[-1] + pd.DateOffset(months=3))

def quarterIndex(timestamp):
    i = 0
    if(timestamp < bin_edges[0] or timestamp >= bin_edges[-1]):
        return -1
    while not (timestamp >= bin_edges[i] and timestamp < bin_edges[i+1]):
        i += 1
    return i


In [10]:
# confirm the index assignment
test = pd.Timestamp("2018-04-02 00:00:00")
print(quarterIndex(test))

37


In [11]:
# assign quarter id to stock data
wrds_ratios["ppln_quarter"] = wrds_ratios["qdate"].apply(lambda quarter:quarterIndex(quarter))
wrds_ratios[["ppln_quarter", "qdate"]].head(2)

,ppln_quarter,qdate
0,3,2009-11-30
1,3,2009-11-30


In [12]:
# assign quarter id to stock data
stock_values["ppln_quarter"] = stock_values["datadate"].apply(lambda quarter:quarterIndex(quarter))
stock_values[["ppln_quarter", "datadate"]].head(2)

,ppln_quarter,datadate
index,,
0,-1,2004-09-01
1,-1,2004-10-01


In [13]:
# assign quarter id to stock data
compustat["ppln_quarter"] = compustat["datadate"].apply(lambda quarter:quarterIndex(quarter))
compustat[["ppln_quarter", "datadate"]].tail(2)

,ppln_quarter,datadate
497473,51,2021-12-31
497474,-1,2023-03-31


In [14]:
# remove out of scope data points
wrds_ratios = wrds_ratios[wrds_ratios["ppln_quarter"] != -1]
compustat = compustat[compustat["ppln_quarter"] != -1]
stock_values = stock_values[stock_values["ppln_quarter"] != -1]

In [15]:
# drop where there is no cusip, which makes them uncomparable
wrds_ratios = wrds_ratios.dropna(subset=["cusip"])
compustat = compustat.dropna(subset=["cusip"])
stock_values = stock_values.dropna(subset=["cusip"])

In [16]:
# adjust cusip to 8 digit cusip
stock_values["ppln_cusip"] = stock_values["cusip"].apply(lambda x:x[:-1])
stock_values.head(1)

,datadate,prcod,prchd,prcld,prccd,cusip,ppln_quarter,ppln_cusip
index,,,,,,,,
92,2009-01-01,0.58,1.06,0.55,0.87,85513Q103,0,85513Q10


In [17]:
# handle cusip as string
compustat["ppln_cusip"] = compustat["cusip"].apply(lambda x:str(x)[:-1])
compustat.head(1)

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsqp,xoptq,xoptqp,xrdq,xsgaq,costat,dvpspq,adjex,ppln_quarter,ppln_cusip
0,1004,2010-02-28,2009,3.0,5,INDL,C,D,STD,AIR,...,0.0,0.0,0.0,NaN,34.091,A,0.0,1.0,4,00036110


In [18]:
# check cusip of WRDS
wrds_ratios.head(1)

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,rd_sale,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,TICKER,cusip,ppln_quarter
0,1004,54594,2009-05-31,2009-11-30,2010-01-31,17.934,1.077,7.981,14.758,15.243,...,0.0,0.0,0.0,-0.043,1.152,2.25,NaN,AIR,00036110,3


## Reduce to one row per Cusip per Quarter

The target is to have one single row per quarter with features. However some quarters and cusips have multiple entries. This is solved by keeping the earliest entry per quarter.

In [19]:
# add exact date of publication for each data point
wrds_ratios["ppln_public_date"] = wrds_ratios["public_date"].apply(lambda x:pd.Timestamp(x))
wrds_ratios.sort_values(by="ppln_public_date", ascending=False).head(1)

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,TICKER,cusip,ppln_quarter,ppln_public_date
572940,349972,15642,2021-12-31,2022-09-30,2022-12-31,NaN,1.545,-2.138,NaN,NaN,...,NaN,NaN,0.014,0.434,NaN,NaN,INDP,45339J10,54,2022-12-31


In [20]:
# longer runtime
# trim down to only one entry per quarter per cusip (WRDS)
clean_ratios = []
cusip_groups = wrds_ratios.groupby(by="cusip")
for cusip, group in cusip_groups:
    quarter_groups = group.groupby(by="ppln_quarter")
    for quarter, q_group in quarter_groups:
        oldest_entry = q_group.sort_values(by="ppln_public_date", ascending=False).iloc[0]
        clean_ratios.append(oldest_entry)

clean_ratios_df = pd.DataFrame(columns=wrds_ratios.columns, data=clean_ratios)
clean_ratios_df.head(1)

,gvkey,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,adv_sale,staff_sale,accrual,ptb,PEG_trailing,divyield,TICKER,cusip,ppln_quarter,ppln_public_date
520968,178698,14945,2014-12-31,2014-12-31,2015-04-30,NaN,0.15,39.025,78.773,78.773,...,0.118,0.411,-0.004,7.629,NaN,NaN,AAC,00030710,23,2015-04-30


In [24]:
# check for duplicate entries in compustat
com_groups = compustat.groupby(by=["ppln_cusip", "ppln_quarter"])
i = 0
j = 0
for name, group in com_groups:
    j += 1
    if len(group.index) != 1:
        i += 1
print("Bins:", j)
print("Bins with more than 1 entry:", i)

Bins: 491549
Bins with more than 1 entry: 442


In [22]:
# longer runtime
# trim down to only one entry per quarter per cusip (compustat)
clean_compustat = []
cusip_groups = compustat.groupby(by="ppln_cusip")
for cusip, group in cusip_groups:
    quarter_groups = group.groupby(by="ppln_quarter")
    for quarter, q_group in quarter_groups:
        entry = q_group.iloc[0]
        clean_compustat.append(entry)
clean_compustat_df = pd.DataFrame(columns=compustat.columns, data=clean_compustat)
clean_compustat_df

,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,xoptepsqp,xoptq,xoptqp,xrdq,xsgaq,costat,dvpspq,adjex,ppln_quarter,ppln_cusip
418526,178698,2012-03-31,2012,1.0,12,INDL,C,D,STD,AACH,...,NaN,0.0,NaN,0.000,NaN,I,0.0,0.01,12,00030710
418527,178698,2012-06-30,2012,2.0,12,INDL,C,D,STD,AACH,...,NaN,0.0,NaN,0.000,NaN,I,0.0,0.01,13,00030710
418528,178698,2012-09-30,2012,3.0,12,INDL,C,D,STD,AACH,...,NaN,0.0,NaN,0.000,NaN,I,0.0,0.01,14,00030710
418529,178698,2012-12-31,2012,4.0,12,INDL,C,D,STD,AACH,...,NaN,0.0,NaN,0.000,NaN,I,0.0,0.01,15,00030710
418530,178698,2013-03-31,2013,1.0,12,INDL,C,D,STD,AACH,...,NaN,0.0,NaN,0.000,NaN,I,0.0,1.00,16,00030710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166009,26340,2021-12-31,2021,4.0,12,INDL,C,D,STD,WVE,...,NaN,NaN,NaN,25.761,NaN,A,0.0,1.00,51,Y9530810
166010,26340,2022-03-31,2022,1.0,12,INDL,C,D,STD,WVE,...,NaN,NaN,NaN,27.470,NaN,A,0.0,1.00,52,Y9530810
166011,26340,2022-06-30,2022,2.0,12,INDL,C,D,STD,WVE,...,NaN,NaN,NaN,29.733,NaN,A,0.0,1.00,53,Y9530810
166012,26340,2022-09-30,2022,3.0,12,INDL,C,D,STD,WVE,...,NaN,NaN,NaN,27.575,NaN,A,0.0,1.00,54,Y9530810


In [25]:
# check for duplicate entries in stock values
value_groups = stock_values.groupby(by=["ppln_cusip", "ppln_quarter"])
i = 0
j = 0
for name, group in value_groups:
    j += 1
    if len(group.index) != 1:
        i += 1
print("Bins:", j)
print("Bins with more than 1 entry:", i)

Bins: 265653
Bins with more than 1 entry: 261338


In [26]:
# longer runtime
# trim down to only one entry per quarter per cusip (stock values)
clean_values = []
cusip_groups = stock_values.groupby(by="ppln_cusip")
for cusip, group in cusip_groups:
    quarter_groups = group.groupby(by="ppln_quarter")
    for quarter, q_group in quarter_groups:
        oldest_entry = q_group.sort_values(by="datadate", ascending=False).iloc[0]
        clean_values.append(oldest_entry)
clean_values_df = pd.DataFrame(columns=stock_values.columns, data=clean_values)
clean_values_df

,datadate,prcod,prchd,prcld,prccd,cusip,ppln_quarter,ppln_cusip
834292,2021-09-01,26.00,32.9900,26.00,29.37,00032Q104,50,00032Q10
834295,2021-12-01,21.28,26.6000,18.75,24.15,00032Q104,51,00032Q10
834298,2022-03-01,17.30,20.0900,16.50,16.97,00032Q104,52,00032Q10
834301,2022-06-01,16.17,17.2700,11.72,12.32,00032Q104,53,00032Q10
834304,2022-09-01,13.52,14.3400,11.66,14.13,00032Q104,54,00032Q10
...,...,...,...,...,...,...,...,...
523591,2021-12-01,3.98,4.0200,3.07,3.14,Y95308105,51,Y9530810
523594,2022-03-01,2.44,2.5950,1.92,2.00,Y95308105,52,Y9530810
523597,2022-06-01,1.38,3.5718,1.25,3.25,Y95308105,53,Y9530810
523600,2022-09-01,2.91,4.1300,2.56,3.61,Y95308105,54,Y9530810


## Join WRDS ans Compustat

In [21]:
print("Unique entries in...")
print("...WRDS Ratios:")
print(len(clean_ratios_df))
print("...Compustat Data:")
print(len(clean_compustat_df))
print("...Stock values:")
print(len(clean_values_df))

Unique entries in...
...WRDS Ratios:
195019
...Compustat Data:
491549
...Stock values:
265653


In [22]:
wrds_and_compustat = clean_ratios_df.merge(
    right=clean_compustat_df,
    right_on=["ppln_cusip", "ppln_quarter"],
    left_on=["cusip", "ppln_quarter"],
    how="inner",
    suffixes=("_wrds","_compustat")
)
wrds_and_compustat.head(1)

,gvkey_wrds,permno,adate,qdate,public_date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,...,xoptepsq,xoptepsqp,xoptq,xoptqp,xrdq,xsgaq,costat,dvpspq,adjex,ppln_cusip
0,178698,14945,2014-12-31,2014-12-31,2015-04-30,NaN,0.15,39.025,78.773,78.773,...,0.0,NaN,0.0,NaN,0.0,NaN,I,0.0,1.0,00030710


## Configure the shift in quarters for the stock value prediction

In [67]:
shift = 1
clean_values_df.describe()

,datadate,prcod,prchd,prcld,prccd,ppln_quarter
count,265653,265653.000000,265653.000000,265653.000000,265653.000000,265653.000000
mean,2016-09-15 17:33:54.718975488,50.308058,55.266786,44.929815,48.837766,30.178440
min,2009-01-01 00:00:00,0.000000,0.000100,0.000000,0.000000,0.000000
25%,2013-03-01 00:00:00,9.433333,10.200000,8.350000,9.320000,16.000000
50%,2016-12-01 00:00:00,21.410000,22.800000,19.710000,21.170000,31.000000
75%,2020-06-01 00:00:00,34.000000,36.250000,31.510000,33.802000,45.000000
max,2022-12-01 00:00:00,208100.000000,208500.000000,184200.000000,192500.000000,55.000000
std,NaN,1296.719609,1414.565328,1155.825246,1246.870126,16.368694


Because stock price values cannot be compared really, we also add some relative stock price measures.

In [79]:
prediction_prices = clean_values_df.reset_index()
prediction_prices["ppln_shift_quarter"] = prediction_prices["ppln_quarter"].apply(lambda x: x - shift)
prediction_prices = prediction_prices.sort_values(by=["ppln_cusip", "ppln_quarter"])


price_shift = []
rel_price_shift = []
for index, row in prediction_prices.iterrows():
    if index >= shift and prediction_prices["ppln_cusip"].iloc[index - shift] == row["ppln_cusip"]:
        #explain all of these variable choices
        pre_prcod = prediction_prices["prcod"].iloc[index - shift]
        price_shift.append(row["prcod"] - pre_prcod)
        rel_price_shift.append((row["prcod"] - pre_prcod)/pre_prcod) #why pre and not row["prcod"]
    else:
        price_shift.append( None )
        rel_price_shift.append( None )
prediction_prices["ppln_price_gain_over_shift"] = price_shift
prediction_prices["ppln_rel_price_gain_over_shift"] = rel_price_shift
prediction_prices = prediction_prices.drop(["index"], axis = 1)
prediction_prices

/var/folders/8y/krd6prkx72780trn94p2wzlw0000gn/T/ipykernel_87739/2922174469.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  rel_price_shift.append((row["prcod"] - pre_prcod)/pre_prcod) #why pre and not row["prcod"]


,datadate,prcod,prchd,prcld,prccd,cusip,ppln_quarter,ppln_cusip,ppln_shift_quarter,ppln_price_gain_over_shift,ppln_rel_price_gain_over_shift
0,2021-09-01,26.00,32.9900,26.00,29.37,00032Q104,50,00032Q10,49,NaN,NaN
1,2021-12-01,21.28,26.6000,18.75,24.15,00032Q104,51,00032Q10,50,-4.72,-0.181538
2,2022-03-01,17.30,20.0900,16.50,16.97,00032Q104,52,00032Q10,51,-3.98,-0.187030
3,2022-06-01,16.17,17.2700,11.72,12.32,00032Q104,53,00032Q10,52,-1.13,-0.065318
4,2022-09-01,13.52,14.3400,11.66,14.13,00032Q104,54,00032Q10,53,-2.65,-0.163884
...,...,...,...,...,...,...,...,...,...,...,...
265648,2021-12-01,3.98,4.0200,3.07,3.14,Y95308105,51,Y9530810,50,-2.29,-0.365231
265649,2022-03-01,2.44,2.5950,1.92,2.00,Y95308105,52,Y9530810,51,-1.54,-0.386935
265650,2022-06-01,1.38,3.5718,1.25,3.25,Y95308105,53,Y9530810,52,-1.06,-0.434426
265651,2022-09-01,2.91,4.1300,2.56,3.61,Y95308105,54,Y9530810,53,1.53,1.108696


## Join the stock prices as features with wrds and compustat

In [81]:
full_feature_df = clean_values_df.merge(
    wrds_and_compustat,
    how="outer",
    left_on=["ppln_quarter","ppln_cusip"],
    right_on=["ppln_quarter","ppln_cusip"],
    suffixes=("_price", "_fund_data")
    )
full_feature_df.describe()


,datadate_price,prcod,prchd,prcld,prccd,ppln_quarter,gvkey_wrds,permno,qdate,CAPEI,...,xopteps12,xoptepsp12,xoptepsq,xoptepsqp,xoptq,xoptqp,xrdq,xsgaq,dvpspq,adjex
count,265653,265653.000000,265653.000000,265653.000000,265653.000000,350893.000000,157170.000000,157170.00000,157170,1.504920e+05,...,43473.0,47188.0,63054.0,47388.0,6.305300e+04,47397.0,72338.000000,134127.000000,157079.000000,157109.000000
mean,2016-09-15 17:33:54.718975488,50.308058,55.266786,44.929815,48.837766,30.054917,65968.362168,56784.96757,2016-09-24 14:32:01.557549056,8.831034e+12,...,0.0,0.0,0.0,0.0,-9.515804e-08,0.0,61.780316,209.030521,0.116925,1.078340
min,2009-01-01 00:00:00,0.000000,0.000100,0.000000,0.000000,0.000000,1004.000000,10001.00000,2010-01-31 00:00:00,-2.910000e+17,...,0.0,0.0,0.0,0.0,-1.300000e-02,0.0,-31.029000,-45.423000,0.000000,0.016700
25%,2013-03-01 00:00:00,9.433333,10.200000,8.350000,9.320000,16.000000,13041.000000,17368.00000,2013-06-30 00:00:00,-4.058000e+00,...,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.661000,8.814000,0.000000,1.000000
50%,2016-12-01 00:00:00,21.410000,22.800000,19.710000,21.170000,31.000000,29734.000000,76052.00000,2016-09-30 00:00:00,1.645700e+01,...,0.0,0.0,0.0,0.0,0.000000e+00,0.0,6.003000,31.500000,0.000000,1.000000
75%,2020-06-01 00:00:00,34.000000,36.250000,31.510000,33.802000,44.000000,132482.000000,87184.00000,2019-12-31 00:00:00,3.045100e+01,...,0.0,0.0,0.0,0.0,0.000000e+00,0.0,21.428250,104.639000,0.130000,1.000000
max,2022-12-01 00:00:00,208100.000000,208500.000000,184200.000000,192500.000000,55.000000,349972.000000,93436.00000,2022-10-31 00:00:00,1.620000e+18,...,0.0,0.0,0.0,0.0,6.000000e-03,0.0,19485.000000,54143.000000,208.790000,28.000000
std,NaN,1296.719609,1414.565328,1155.825246,1246.870126,16.016462,67417.195225,32660.72228,NaN,4.242822e+15,...,0.0,0.0,0.0,0.0,5.715886e-05,0.0,380.512958,991.382658,0.681411,0.696075
